In [1]:
from PIL import Image
import numpy as np
import glob
from collections import Counter
from resizeimage import resizeimage
import re

STANDARD_SIZE=(640, 480)
TRAIN_IMAGES=88
TEST_IMAGES=21
IMG_ROW=224
IMG_COL=168

In [108]:
#Define function
def img_to_matrix(filename, verbose=False):
    img=Image.open(filename)
    if verbose==True:
        print("Changing size from %s to %s " % (str(img.size), str(STANDARD_SIZE)))
    img=img.resize(STANDARD_SIZE)
    img=list(img.getdata())
    img=map(list, img)
    img=np.array(img)
    return img

def flatten_image(img):
    s=img.shape[0]*img.shape[1]
    img_wide=img.reshape(1, s)
    return img_wide[0]

def extract_class(list_of_labels):
    list_classes=[]
    for label in list_of_labels:
        list_classes.append(label.split('\\')[1])
    return list_classes

def convert_to_matrix(img):
    img=np.array(img.getdata()).astype(np.float32).reshape((3, img.size[0], img.size[1]))
    return img

def pre_process_image(img):
    img = resizeimage.resize('thumbnail', img, [330, 220])
    img = resizeimage.resize_crop(img, [330, 220])
    #img = resizeimage.resize_width(img, 224)
    #img = resizeimage.resize('thumbnail', img, [224, 168])
    return img

def return_array_classes(namelist):
    classes=[]
    for car_model in namelist:
        classes.append(int(''.join(re.findall('[0-9]+', car_model)))) #Convert list to string
    return np.asarray(classes)

In [45]:
img=Image.open('est.jpg')
a = np.array(img.getdata()).astype(np.float32).reshape((3, img.size[0], img.size[1]))

In [48]:
#Reading Images
images_list=[]
label_list=[]
for filename in glob.glob('Corrolla Vision Test\*\*.jpg'):
    im=Image.open(filename)
    im_pre_process=pre_process_image(im)
    im_matrix=convert_to_matrix(im_pre_process)
    images_list.append(im_matrix)
    label_list.append(filename)

In [59]:
X_train=array_images[0:88,:,:,:]
X_test=array_images[88:,:,:,:]

In [60]:
test_images=extract_class(label_list)

In [109]:
Y_train=test_images[0:88]
Y_test=test_images[88:]

In [110]:
Y_test_raw=[]
for item in label_list[88:]:
    Y_test_raw.append('Toyota - Corrolla - '+item.split('\\')[2][0:4]+'.jpg')

In [113]:
Y_train=return_array_classes(Y_train)

In [117]:
Y_test=return_array_classes(Y_test_raw)

In [118]:
Y_test

array([1996, 2003, 2008, 2008, 2009, 2009, 2010, 2010, 2011, 2011, 2012,
       2012, 2013, 2013, 2014, 2014, 2014, 2015, 2015, 2016, 2016])

In [119]:
Y_train

array([2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2009,
       2009, 2009, 2009, 2009, 2009, 2009, 2009, 2009, 2009, 2009, 2009,
       2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2011,
       2011, 2011, 2011, 2011, 2011, 2011, 2011, 2012, 2012, 2012, 2012,
       2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2013, 2013, 2013,
       2013, 2013, 2013, 2013, 2013, 2013, 2014, 2014, 2014, 2014, 2014,
       2014, 2014, 2014, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015,
       2015, 2015, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016])

In [121]:
Y_test

array([2008, 2008, 2009, 2009, 2010, 2010, 2011, 2011, 2012, 2012, 2013,
       2013, 2014, 2014, 2014, 2015, 2015, 2016, 2016])

In [123]:
X_test.shape

(21L, 3L, 330L, 220L)

In [124]:
Y_test.shape

(19L,)

In [125]:
X_test=X_test[2:]

In [126]:
X_test.shape

(19L, 3L, 330L, 220L)

In [69]:
Y_test[0]

'Toyota - Corrolla - 2008.jpg'

In [127]:
np.save('X_train.npy', X_train)
np.save('Y_train.npy', Y_train)
np.save('X_test.npy', X_test)
np.save('Y_test.npy', Y_test)

In [58]:
def load_data():
    X_train=np.load('X_train.npy')
    Y_train=np.load('Y_train.npy')
    X_test=np.load('X_test.npy')
    Y_test=np.load('Y_test.npy')
    Y_train_uniques, Y_train_ids = np.unique(Y_train, return_inverse=True)
    Y_test_uniques, Y_test_ids = np.unique(Y_test, return_inverse=True)
    Y_train = np_utils.to_categorical(Y_train_ids, nb_classes)
    Y_test = np_utils.to_categorical(Y_test_ids, nb_classes)
    return X_train, Y_train, X_test, Y_test

In [28]:
#To bring classes in 0-9 range
#Y_train=np.fmod(Y_train, 10)
#Y_test=np.fmod(Y_test, 10)

In [30]:
import keras

In [31]:
from __future__ import print_function
#from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

In [111]:
batch_size = 22
nb_classes = 9
nb_epoch = 10
data_augmentation = True

In [112]:
img_rows, img_cols = 320, 220
# the CIFAR10 images are RGB
img_channels = 3

# the data, shuffled and split between train and test sets
X_train, Y_train, X_test, Y_test = load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (88L, 3L, 330L, 220L)
88 train samples
19 test samples


In [113]:
model = Sequential()

model.add(Convolution2D(20, 9, 7, border_mode='same',
                        input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(20, 9, 7))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Convolution2D(30, 10, 7, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(30, 9, 5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Convolution2D(40, 7, 4, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(40, 7, 5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Convolution2D(40, 7, 5, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(40, 6, 5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [114]:

# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [115]:

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=True,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))

Using real-time data augmentation.
Epoch 1/10
66/88 [=====================>........] - ETA: 13s - loss: 4.4537 - acc: 0.1212